In [1]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [2]:
#del sys.modules["utilities"]
from utilities import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [5]:
df_feats_surgeryprocess = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_SurgeryProcess.pkl','rb'))
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))
df_feats_icd_transform = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_icd_transform.pkl','rb'))
df_preor_ICULOS = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_preor_ICULOS.pkl','rb'))
df_feats_delirium = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_delirium.pkl','rb'))

df_icu_graph_features = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_icu_graph_features.pkl','rb'))

df_feats_ppg_train_all = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_train_separate.pkl','rb'))
df_feats_ppg_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_val_separate.pkl','rb'))
df_feats_ppg_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_test.pkl','rb'))


feats_n2v_all_train = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_train.pkl','rb'))
feats_n2v_all_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_val.pkl','rb'))
feats_n2v_all_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_test.pkl','rb'))



Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels_death_30d.pkl','rb'))

In [6]:
Outcome_labels.head()

,ENCRYPTED_HOSP_ENCOUNTER,IF_died_postsurg30d
0,b'\x16\x7f\x17:\xdeX[\xb4\xf3\x80.\xaa\xd6OU',0
1,b'\x16\x7f\x17:\xdeXZ}\xcf=\xe5\x1a.w$',0
2,b'\x16\x7f\x17:\xdeXZ}\xcf=\xe6\xd2\xa1\xe6\xdd',0
3,b'\x16\x7f\x17:\xdeXZ}\xc3sQ\xd3\xb9Y\x0f',0
4,b'\x16\x7f\x17:\xdeXZ}\xcf>%n\x11!V',0


In [7]:
df_feats_generalclinical_process_icug = df_feats_clinical.merge(df_feats_icd_transform,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_preor_ICULOS,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_feats_delirium,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_feats_surgeryprocess,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left').merge(df_icu_graph_features,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left')

In [8]:
Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','IF_died_postsurg30d']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','IF_died_postsurg30d']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','IF_died_postsurg30d']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [9]:
df_feats_generalclinical_process_ppg_icug_train = df_feats_generalclinical_process_icug[df_feats_generalclinical_process_icug['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_train_all,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_train,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_generalclinical_process_ppg_icug_val = df_feats_generalclinical_process_icug[df_feats_generalclinical_process_icug['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_val,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_generalclinical_process_ppg_icug_test = df_feats_generalclinical_process_icug[df_feats_generalclinical_process_icug['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_test,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


len(df_feats_generalclinical_process_ppg_icug_train)+len(df_feats_generalclinical_process_ppg_icug_val)+len(df_feats_generalclinical_process_ppg_icug_test)-len(df_feats_generalclinical_process_icug)

0

In [10]:
y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_test = Outcome_labels_test

X_train = y_train[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_generalclinical_process_ppg_icug_train,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)
X_val = y_val[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_generalclinical_process_ppg_icug_val,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_test = y_test[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_generalclinical_process_ppg_icug_test,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_train_val = pd.concat([X_train,X_val]).reset_index(drop=True)
y_train_val = pd.concat([y_train,y_val]).reset_index(drop=True)

In [11]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 4520),
 (13665, 4520),
 (122983, 4520),
 (13664, 4520),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [12]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [13]:
del sys.modules["utilities"]
from utilities import *

In [14]:
#experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[2023])

In [15]:
np.random.seed(666)

In [16]:
a = np.zeros(len(X_train))-1 
b = np.ones(len(X_val)) 
pds = PredefinedSplit(np.concatenate([a,b]))

In [17]:
pipe = Pipeline(steps=[
    ("sampler",RandomUnderSampler(random_state=666)),
                       ("imputer", SimpleImputer(missing_values=np.nan, strategy='mean')),
                       ("scaler", MinMaxScaler()),
                       ("classifier", MLPClassifier()) #default 3-layer mlp
                        ])

In [21]:
clf = BayesSearchCV(estimator = pipe,
                   search_spaces={
                    'classifier__alpha':Real(0.0001,0.99,'log-uniform'),
                    'classifier__learning_rate': Categorical(['constant','adaptive']),
                    'classifier__early_stopping': Categorical([True,False])
                   },
                   n_iter = 30,
                   cv=pds,
                   scoring = 'f1',
                   n_jobs=24,
                   refit=True,
                   verbose=4,
                   random_state=666
                   ) 

In [22]:
clf.fit(X_train_val.iloc[:,1:],y_train_val.iloc[:,1:].values.ravel())

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits


/apps/python/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Fitting 1 folds for each of 1 candidates, totalling 1 fits


/apps/python/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV 1/1] END classifier__alpha=0.037999333430404396, classifier__early_stopping=True, classifier__learning_rate=constant;, score=0.207 total time=  29.9s
[CV 1/1] END classif

/apps/python/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/1] END classifier__alpha=0.07514813566541474, classifier__early_stopping=True, classifier__learning_rate=adaptive;, score=0.191 total time=  25.1s
[CV 1/1] END classifier__alpha=0.99, classifier__early_stopping=True, classifier__learning_rate=constant;, score=0.197 total time=  28.8s
[CV 1/1] END classifier__alpha=0.00010163622259753977, classifier__early_stopping=True, classifier__learning_rate=constant;, score=0.224 total time=  23.1s
[CV 1/1] END classifier__alpha=0.00010005431727017637, classifier__early_stopping=True, classifier__learning_rate=adaptive;, score=0.209 total time=  20.8s


BayesSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
              estimator=Pipeline(steps=[('sampler',
                                         RandomUnderSampler(random_state=666)),
                                        ('imputer', SimpleImputer()),
                                        ('scaler', MinMaxScaler()),
                                        ('classifier', MLPClassifier())]),
              n_iter=30, n_jobs=24, random_state=666, scoring='f1',
              search_spaces={'classifier__alpha': Real(low=0.0001, high=0.99, prior='log-uniform', transform='normalize'),
                             'classifier__early_stopping': Categorical(categories=(True, False), prior=None),
                             'classifier__learning_rate': Categorical(categories=('constant', 'adaptive'), prior=None)},
              verbose=4)

In [23]:
print(clf.best_params_)
best_m = clf.best_estimator_
pred = best_m.predict(X_test.iloc[:,1:])
pred_prob = best_m.predict_proba(X_test.iloc[:,1:])[:,1]
print(min(pred_prob),max(pred_prob))

print('Best Validation AUROC: {}'.format(clf.best_score_))

print('Test AUROC: {}'.format(roc_auc_score(y_test.iloc[:,1:], pred_prob)))
print('Test F1-score: {}'.format(f1_score(y_test.iloc[:,1:], pred)))
print('Test Recall: {}'.format(recall_score(y_test.iloc[:,1:], pred)))
print('Test Precision: {}'.format(precision_score(y_test.iloc[:,1:], pred)))
print('Test Accuracy: {}'.format(accuracy_score(y_test.iloc[:,1:], pred)))

OrderedDict([('classifier__alpha', 0.00010092562980601905), ('classifier__early_stopping', True), ('classifier__learning_rate', 'adaptive')])
5.276791963716105e-06 0.9990760501204345
Best Validation AUROC: 0.22702702702702704
Test AUROC: 0.883410267662628
Test F1-score: 0.21715526601520085
Test Recall: 0.8247422680412371
Test Precision: 0.12503907471084713
Test Accuracy: 0.7889344262295082
[CV 1/1] END classifier__alpha=0.99, classifier__early_stopping=True, classifier__learning_rate=constant;, score=0.219 total time=  20.5s
[CV 1/1] END classifier__alpha=0.00010113702922946334, classifier__early_stopping=True, classifier__learning_rate=constant;, score=0.219 total time=  22.0s
[CV 1/1] END classifier__alpha=0.00010020525231328869, classifier__early_stopping=False, classifier__learning_rate=adaptive;, score=0.150 total time= 4.6min
